In [3]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from helpers import util
from helpers import config
import json
import os
import warnings
warnings.filterwarnings("ignore")

In [ ]:
project = hopsworks.login(engine="python")
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

# This line will fail if you have not registered the AQICN_API_KEY as a secret in Hopsworks
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
sensors_str = secrets.get_secret("SENSORS_JSON").value
sensors_data = json.loads(sensors_str)

today = datetime.date.today()

2025-11-09 23:36:44,292 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-11-09 23:36:44,301 INFO: Initializing external client
2025-11-09 23:36:44,301 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-09 23:36:45,554 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1272018


In [9]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

In [ ]:
import requests
import pandas as pd

aq_dfs = []

for location in sensors_data:
    aqicn_url = location['aqicn_url']
    country = location['country']
    city = location['city']
    street = location['street']
    aq_location_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)
    aq_location_today_df['lagged_1'] = aq_location_today_df['pm25'].shift(1)
    aq_location_today_df['lagged_2'] = aq_location_today_df['pm25'].rolling(window=2).mean().shift(1)
    aq_location_today_df['lagged_3'] = aq_location_today_df['pm25'].rolling(window=3).mean().shift(1)
    aq_dfs.append(aq_location_today_df)

aq_today_df = pd.concat(aq_dfs, ignore_index=True) 

In [ ]:
aq_today_df.info()

In [ ]:
weather_dfs = []

for location in sensors_data:
    city = location['city']
    latitude = location['latitude']
    longitude = location['longitude']
    hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
    hourly_df = hourly_df.set_index('date')

    # We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
    # We only want the daily weather data, so only get weather at 12:00
    daily_location_df = hourly_df.between_time('11:59', '12:01')
    daily_location_df = daily_location_df.reset_index()
    daily_location_df['date'] = pd.to_datetime(daily_location_df['date']).dt.date
    daily_location_df['date'] = pd.to_datetime(daily_location_df['date'])
    daily_location_df['city'] = city
    weather_dfs.append(daily_location_df)

daily_df = pd.concat(weather_dfs, ignore_index=True) 

In [ ]:
daily_df.info()
daily_df.head(500)

In [24]:
# Insert new data
air_quality_fg.insert(aq_today_df)
weather_fg.insert(daily_df, wait=True)

FeatureStoreException: Features are not compatible with Feature Group schema: 
 - lagged_1 (type: 'float') is missing from input dataframe.
 - lagged_2 (type: 'double') is missing from input dataframe.
 - lagged_3 (type: 'double') is missing from input dataframe.
Note that feature (or column) names are case insensitive and spaces are automatically replaced with underscores.